In [ ]:
import plotly.express as px

from core.pandas_utils import *
from core.stats_utils import *
from transform.raw_results.tesla_results import get_results
from transform.raw_results.get_tesla_soh_readouts import get_aviloo_soh_readouts

In [ ]:
results = get_results()

In [ ]:
BASE_SLOPE = 0.8 / 1e4 # base soh loss per kilometer

def evaluate_esimtations(results:DF, soh_cols:list[str]) -> DF:
    results = results.merge(get_aviloo_soh_readouts()[["vin", "soh_readout"]], on="vin", how="left")
    return DF({soh_col: evaluate_single_estimation(results, soh_col)} for soh_col in soh_cols)

def evaluate_single_estimation(results:DF, soh_col:DF, aviloo_soh_readouts:DF) -> DF:
    results_grp = results.groupby("vin")
    lr_params:DF = results_grp.apply(lr_params_as_series, "odometer", "soh_col", include_groups=False).reset_index()
    return (
        results
        .merge(lr_params, on="vin", how="left")
        .eval("lr_soh = odometer * slope + intercept")
        .eval("soh_lr_diff = soh - lr_soh")
        .groupby("vin")
        .agg({
            "soh_lr_diff": "std",
            soh_col: "median",
        })
        .merge(lr_params, on="vin", how="left")# yes I know I already merged into it..,
        .merge(aviloo_soh_readouts[["vin", "soh_readout"]], on="vin", how="left")
        .assign(abs_soh_residual=lambda df: (df[soh_col] - df["soh_readout"]).abs()) #Have to use assign because .abs() does not work in eval call, idk why...
        .assign(abs_diff_to_base_trendline=lambda df: (df["soh_lr_diff"] - BASE_SLOPE).abs())
        .agg({"abs_soh_residual": "meab", "soh_lr_diff": "mean", "abs_diff_to_base_trendline": "mean", })
    )